In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras_hub



ImportError: cannot import name 'ops' from 'keras' (/usr/local/lib/python3.11/dist-packages/keras/__init__.py)

# Daten aufbereiten

In [ ]:
raw_train_ds, raw_val_ds = tfds.load('imdb_reviews', split=['train', 'test'])

In [ ]:
for x in raw_train_ds.take(1):
    print(x)

In [ ]:
train_ds = raw_train_ds.map(lambda x: (x['text'], x['label']))
for (x, y) in train_ds.take(1):
    print(x)
    print(y)

In [ ]:
train_ds = train_ds.shuffle(1000).batch(128).prefetch(tf.data.AUTOTUNE)
val_ds = raw_val_ds.shuffle(1000).take(2500).map(lambda x: (x['text'], x['label'])).batch(128).prefetch(
        tf.data.AUTOTUNE
        )

# Erster Versuch

In [ ]:
bert_name = "bert_tiny_en_uncased"
tokenizer = keras_hub.models.BertTokenizer.from_preset(bert_name)
text = "This is a test"
text_tk = tokenizer(text)
print(text_tk)

In [ ]:
preprocessor = keras_hub.models.BertPreprocessor.from_preset(bert_name)
backbone = keras_hub.models.BertBackbone.from_preset(bert_name)

In [ ]:
inp = preprocessor(["hallo", "this is a test"])
y = backbone(inp)
print(y.keys())
print(y['pooled_output'].shape)
print(y['sequence_output'].shape)

In [ ]:
backbone.trainable = False

input = tf.keras.Input(shape=(), dtype=tf.string, name='text')
output = backbone(preprocessor(input))
net = output['pooled_output']
#net = output['sequence_output'][:, -1, :]
# net = tf.keras.layers.Dropout(0.1)(net)
net = tf.keras.layers.Dense(512, activation='selu')(net)
net = tf.keras.layers.Dense(512, activation='relu')(net)
net = tf.keras.layers.Dense(128, activation='selu')(net)
net = tf.keras.layers.Dense(1, activation='sigmoid')(net)
model = tf.keras.Model(input, net)
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        patience=2,
        monitor='val_loss',
        min_delta=0.003,
        restore_best_weights=True
        )
model.fit(train_ds, epochs=20, validation_data=val_ds, verbose=2, callbacks=[early_stopping_cb])

In [ ]:
model.save("models/bert_tiny_en_uncased_finetuned.keras")


# Zweiter Versuch

In [ ]:
preprocessor2 = keras_hub.models.BertPreprocessor.from_preset(bert_name)
backbone2 = keras_hub.models.BertBackbone.from_preset(bert_name)

input2 = tf.keras.Input(shape=(), dtype=tf.string, name='text')
output2 = backbone2(preprocessor2(input2))
net2 = output2['pooled_output']
#net = output['sequence_output'][:, -1, :]
# net = tf.keras.layers.Dropout(0.1)(net)
#net2 = tf.keras.layers.Dense(1, activation='sigmoid')(net2)
net2 = tf.keras.layers.Dense(2, activation='softmax')(net2)
model2 = tf.keras.Model(input2, net2)
model2.summary()

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model2.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        patience=2,
        monitor='val_loss',
        min_delta=0.003,
        restore_best_weights=True
        )
model2.fit(train_ds, epochs=20, validation_data=val_ds, verbose=2, callbacks=[early_stopping_cb])

In [ ]:
model2.save("models/bert_tiny_en_uncased_full_finetuned_2.keras")

# Dritter Versuch

In [ ]:
classifier = keras_hub.models.TextClassifier.from_preset(bert_name, sequence_lengths=1024, num_classes=2)
classifier.summary()

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        patience=2,
        monitor='val_loss',
        min_delta=0.003,
        restore_best_weights=True
        )
classifier.fit(train_ds, epochs=20, validation_data=val_ds, verbose=2, callbacks=[early_stopping_cb])
classifier.save("models/classifier_bert_tiny_en_uncased_1024_2_classifier.keras")